# IMPORT ED AUTENTICAZIONE

In [19]:
import os 
import json
import pprint
import tweepy

In [20]:
#Twitter API credentials
api_key = "VlNL2LO3nPv8sN7HoTwiHPWRJ"
api_secret = "0x66wQ6hJrEF0sRsDDBk5F84GPdZrW7ofrLAHG9d4VOKpuNDU8"
access_token = "4878912292-yHYq0PzMFAzOgHmQ5CQG0Ls7tiqGOIcV7jKOxzu"
access_secret = "9uJKXlMqXrRtn9TxqtAL2SBv8fLGVZH4bu6IcHhKCDbMt"
bearer_token = "AAAAAAAAAAAAAAAAAAAAAA2xVwEAAAAAWWRrvmkGwNuTP%2FMltp7z1%2F1Wh5A%3DuCQUQaeBJ9EEmLrOPUGoDZ4bI6s1fEzguiNNOyceGGFjhKmFPn"

In [21]:
auth = tweepy.OAuthHandler(api_key, api_secret)
auth.set_access_token(access_token, access_secret)
api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)
if(api.verify_credentials):
    print ('Authentication completed successfully!')

Authentication completed successfully!


# GESTIONE JSON

In [22]:
data_folder = "data"

def serialize_json(folder, filename, data):
    if not os.path.exists(folder):
        os.makedirs(folder, exist_ok=True)
    with open(f"{folder}/{filename}", 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent=4)
        f.close()
    print(f"Data serialized to path: {folder}/{filename}")

def read_json(path):
    if os.path.exists(path):
        with open(path, "r", encoding="utf8") as file:
            data = json.load(file)
        print(f"Data read from path: {path}")
        return data
    else:
        print(f"No data found at path: {path}")
        return {}


# FOLLOWERS

In [ ]:
utenti = ["mizzaro", "Miccighel_"]
followers = {}
quantity = 5

for utente in utenti:
    
    print(f"Processing User {utente}")
    
    followers_of_utente = []

    while len(followers_of_utente)<5:
        item=tweepy.Cursor(api.followers,screen_name=utente,skip_status=True,include_user_entities=False).items(1).next()
        json_data = item._json
        
        found_follower = {}
        found_follower["id"] = json_data["id"]
        found_follower["name"] = json_data["name"]
        found_follower["screen_name"] = json_data["screen_name"]
        found_follower["location"] = json_data["location"]
        found_follower["followers_count"] = json_data["followers_count"]
        found_follower["friends_count"] = json_data["friends_count"]

        if found_follower not in followers_of_utente and found_follower["followers_count"]<1000 and found_follower["friends_count"]<1000:
            followers_of_utente.append(found_follower)
            
    followers[utente] = followers_of_utente
    print(f"Found {len(followers[utente])} followers for user {utente}")
    
serialize_json(data_folder, "utenti_followers.json", followers)

Processing User mizzaro


Rate limit reached. Sleeping for: 826


In [25]:
followers = read_json(f"{data_folder}/utenti_followers.json")
# (Pretty) print the first three followers of Iron Man
import pprint
pp = pprint.PrettyPrinter()
pp.pprint(followers['mizzaro'][:3])

Data read from path: data/utenti_followers.json
[]


# ESEMPI

# 1 - Scaricate i dettagli del profilo di Mizzaro

In [26]:
utente = "mizzaro"
raw_data = api.get_user(utente)
json_data = raw_data._json
profile_data = {}
profile_data["id"] = json_data["id"]
profile_data["name"] = json_data["name"]
profile_data["screen_name"] = json_data["screen_name"]
profile_data["location"] = json_data["location"]
profile_data["description"] = json_data["description"]
profile_data["followers_count"] = json_data["followers_count"]
serialize_json(data_folder, "1_mizzaro_profile_data.json", profile_data)
pp.pprint(profile_data)

Data serialized to path: data/1_mizzaro_profile_data.json
{'description': '',
 'followers_count': 162,
 'id': 18932422,
 'location': '',
 'name': 'mizzaro',
 'screen_name': 'mizzaro'}


# 2 - Scaricate gli ultimi 10 tweet di Mizzaro

In [27]:
utente = "mizzaro"
tweets = []
for item in tweepy.Cursor(
        api.user_timeline, 
        screen_name=utente,
        tweet_mode="extended"
    ).items(10):
    raw_tweet = item._json
    tweet = {}
    tweet["id"]= raw_tweet["id"]
    tweet["created_at"] = raw_tweet["created_at"]
    tweet["full_text"] = raw_tweet["full_text"],
    tweet["favorite_count"]= raw_tweet["favorite_count"]
    tweets.append(tweet)
serialize_json(data_folder, "2_mizzaro_recent_tweets.json", tweets)
pp.pprint(tweets)


Data serialized to path: data/2_mizzaro_recent_tweets.json
[{'created_at': 'Thu Nov 11 18:44:44 +0000 2021',
  'favorite_count': 0,
  'full_text': ('@JulioGonzalo1 @lintool But you did it!',),
  'id': 1458868311777763333},
 {'created_at': 'Mon Oct 18 07:49:11 +0000 2021',
  'favorite_count': 0,
  'full_text': ('RT @eglu81: "We argue that bias is part of human nature, and '
                'that it should be managed rather than removed." \n'
                '\n'
                'From our position paper at #…',),
  'id': 1450006026745491459},
 {'created_at': 'Mon Oct 04 22:04:34 +0000 2021',
  'favorite_count': 0,
  'full_text': ('RT @caslernoel: Anti-vaxxers react to FB outage: ‘how am I '
                'supposed to do my own research now?’',),
  'id': 1445147863453995013},
 {'created_at': 'Mon Oct 04 21:39:32 +0000 2021',
  'favorite_count': 0,
  'full_text': ('RT @AcademicsSay: academia is wild because one day you get '
                'behind on writing and then you stay that way y

# 3 - Scaricate 20 tweet preferiti da Mizzaro

In [28]:
utente = "mizzaro"
raw_favorites = api.favorites(
    screen_name=utente,
    tweet_mode="extended"
)
tweets = []
for raw_tweet in raw_favorites:
    json_tweet = raw_tweet._json
    tweet = {}
    tweet["id"]= json_tweet["id"]
    tweet["created_at"] = json_tweet["created_at"]
    tweet["full_text"] = json_tweet["full_text"],
    tweet["favorite_count"]= json_tweet["favorite_count"]
    tweets.append(tweet)
serialize_json(data_folder, "3_mizzaro_favorite_tweets.json", tweets)
pp.pprint(tweets)


Data serialized to path: data/3_mizzaro_favorite_tweets.json
[{'created_at': 'Thu Nov 11 18:11:13 +0000 2021',
  'favorite_count': 2,
  'full_text': ('@lintool "reproducibility is hard, especially after the first '
                'occurrence" (@mizzaro)',),
  'id': 1458859876432723980},
 {'created_at': 'Tue Nov 09 17:57:21 +0000 2021',
  'favorite_count': 232,
  'full_text': ('Some news from my side. Totally excited and honored. '
                'https://t.co/T4O3ECjIsu',),
  'id': 1458131610436579340},
 {'created_at': 'Sun Nov 07 09:14:29 +0000 2021',
  'favorite_count': 11628,
  'full_text': ('La differenza tra Berlusconi e Renzi è che Berlusconi faceva '
                'di tutto per starci simpatico e il Fatto lavorava sodo per '
                'farcelo stare antipatico, mentre Renzi lavora come un matto '
                'per star sul cazzo a tutti e il Fatto cerca in tutti i modi '
                'di farcelo stare simpatico.',),
  'id': 1457275251105144833},
 {'created_at': '

# 4 - Scaricare 1 retwittatore di un ciascuno dei tweet preferiti 

In [29]:
mizzaro_favorite_tweets = read_json("data/3_mizzaro_favorite_tweets.json")
utente = "mizzaro"
retweeters = {}
for tweet in mizzaro_favorite_tweets:
    print(f"Processing tweet with id: {tweet['id']}")
    for item in tweepy.Cursor(
            api.retweeters, 
            id=tweet["id"],
        ).items(1):
        print(f"Retweeter ID: {item}")    
        json_data = api.get_user(id=item)._json
        if json_data["id"] not in retweeters.keys():
            profile_data = {}
            profile_data["id"] = json_data["id"]
            profile_data["name"] = json_data["name"]
            profile_data["screen_name"] = json_data["screen_name"]
            profile_data["location"] = json_data["location"]
            profile_data["description"] = json_data["description"]
            profile_data["followers_count"] = json_data["followers_count"]
            retweeters[profile_data["id"]] = profile_data
serialize_json(data_folder, "4_mizzaro_favorite_tweets_retweeters.json", retweeters)
pp.pprint(retweeters)
     

Data read from path: data/3_mizzaro_favorite_tweets.json
Processing tweet with id: 1458859876432723980
Processing tweet with id: 1458131610436579340
Retweeter ID: 26711278
Processing tweet with id: 1457275251105144833
Retweeter ID: 713663420827049984
Processing tweet with id: 1457174095548887040
Retweeter ID: 2599246273


KeyboardInterrupt: 